In [1]:
import os, sys
import unidecode
import re
import json
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Ce notebook analyse les réponses aux questions en cherchant des liens entre les mots les plus utilisés, dans le but de comprendre les demandes les plus récurrentes des francais.

Dans un premier temps, les réponses sont préprocessées et transformées en liste de mots-clés. Puis, pour chaque réponse, úne analyse basique de la proximité des mots est réalisée. Bien que simple, cette methode permet de faire apparaitre des thèmes récurrents et de comprendre la teneur générale de ces demandes.

Les trois métriques utilisées sont :
+ co-occurence des mots : $co-occurence(i, j) = \sum_{users} 1_{i \in user} 1_{j \in user}$
+ confiance : $confiance(i -> j) = P(j | i)$
+ lift : $lift(i, j) = \frac{P(i \cap j)}{P(i)P(j)}$

La co-occurence permet d'évaluer les mots présents de manière commune, et donne une première idée des thèmes généraux.

La confiance donne une idée de la présence de $j$ quand $i$ est déjà présent : il surpondère les mots peu présents par rapport à la co-occurence.

Le lift estime le surplus de présence commune de $i$ et $j$ par rapport au cas où $i$ et $j$ sont indépendants. Cela permet de trouver des relations de causalité plus intéressantes, notamment dans des cas où certains mots sont peu présents mais très liés.

Pour chaque question :
+ On récupère les mots les plus employés sur ce thème
+ Pour chacun de ces mots, on regarde pour chaque métrique les mots les plus liés, ce qui donne une bonne idée des liens et des demandes

On peut ensuite raffiner l'analyse sur des mots moins présents mais porteurs de sens.

In [2]:
# function definitions

def compute_similarities(question_name, df_res):
    qu = df_res[question_name].dropna()
    df_qu_one_hot = compute_one_hot(qu)
    df_cov = compute_covariance(df_qu_one_hot)
    df_confidence = compute_confidence(df_cov)
    df_lift = compute_lift(df_cov, df_qu_one_hot)
    
    return qu, df_qu_one_hot, df_cov, df_confidence, df_lift
    
def compute_one_hot(qu):
    qu_words = []
    for i in qu:
        qu_words.extend(i)
    qu_words = list(set(qu_words))
    dic_qu_words = {word:i for i, word in enumerate(qu_words)}
    dic_qu_words_inverse = {i:word for i, word in enumerate(qu_words)}
    
    qu_one_hot = np.zeros((len(qu), len(qu_words)), dtype=bool)
    for i, li_word in enumerate(qu):
        for word in li_word:
            qu_one_hot[i, dic_qu_words[word]] = True
            
    df_qu_one_hot = pd.DataFrame(columns=qu_words, data=qu_one_hot).to_sparse(fill_value=False)

    
    temp_mask = np.sum(df_qu_one_hot.values, axis=0, dtype=np.uint16) > 250
    words_enough = df_qu_one_hot.columns.values[temp_mask]
    df_qu_one_hot = df_qu_one_hot[words_enough].to_sparse(fill_value=0)
    
    del temp_mask
    del qu_one_hot
    gc.collect()
    
    return df_qu_one_hot

def compute_covariance(df_qu_one_hot):
    N = df_qu_one_hot.shape[1]
    
    cov = np.zeros((N, N), dtype=np.uint16)
    np.matmul(df_qu_one_hot.values.T, df_qu_one_hot.values, out=cov)
    
    df_cov = pd.DataFrame(columns=df_qu_one_hot.columns, index=df_qu_one_hot.columns, data=cov) * (cov > 100)
    del cov
    gc.collect()
    
    return df_cov

def compute_confidence(df_cov):
    df_confidence = np.diag(1 / np.diag(df_cov)).dot(df_cov).astype(np.float32)
    df_confidence = pd.DataFrame(index=df_cov.index, columns=df_cov.index, data=df_confidence)
    return df_confidence

def compute_lift(df_cov, df_qu_one_hot):
    inv_diag_cov = 1 / np.diag(df_cov.values)
    lift = inv_diag_cov.reshape((-1, 1)) * df_cov.values * inv_diag_cov.reshape((1, -1))
    df_lift = pd.DataFrame(columns=df_qu_one_hot.columns,
                             index=df_qu_one_hot.columns,
                             data=lift).replace(0.0, np.nan).astype(np.float32)
    del lift
    del inv_diag_cov
    gc.collect()
    
    return df_lift

# Loading of files and preprocessing

In [3]:
PATH_TO_FILE = "./proposals_cleaned/fiscalite.json"

In [4]:
with open(PATH_TO_FILE, "r") as f:
    raw_json = [s.strip() for s in f.readlines()]
    
parsed_json = [json.loads(raw_json[i]) for i in range(len(raw_json))]
del raw_json

In [5]:
col = ["titre"]
for ques in parsed_json[0]["responses"]:
    col.append(ques["question"]["id"]) 
        
data = []
for elt in parsed_json:
    resp = {"titre" : elt["titre"]}
    for ques in elt["responses"]:
        try:
            text = json.loads((ques["value"]))
            resp[ques["question"]["title"]] = ''
            if text["labels"] != []:
                resp[ques["question"]["title"]] += ", ".join(text["labels"])
            if text["other"] is not None:
                resp[ques["question"]["title"]] += ", " + text["other"]
            if resp[ques["question"]["title"]] == '':
                resp[ques["question"]["title"]] = np.nan
        except:
            if ques["value"] is not None and ques["value"][0] != "{":
                resp[ques["question"]["title"]] = ques["value"]
            else:
                resp[ques["question"]["title"]] = np.nan
    data.append(resp)

In [6]:
res = pd.DataFrame.from_dict(data)
del data
gc.collect()

0

In [7]:
res.notna().mean(axis=0).sort_values(ascending=False)

titre                                                                                                                                 1.000000
Afin de financer les dépenses sociales, faut-il selon vous...                                                                         0.827525
Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?                                                          0.824925
Quels sont selon vous les impôts qu'il faut baisser en priorité ?                                                                     0.754784
Y a-t-il d'autres points sur les impôts et les dépenses sur lesquels vous souhaiteriez vous exprimer ?                                0.698169
Pour quelle(s) politique(s) publique(s) ou pour quels domaines d'action publique, seriez-vous prêts à payer plus d'impôts ?           0.698126
Quelles sont toutes les choses qui pourraient être faites pour améliorer l'information des citoyens sur l'utilisation des impôts ?    0.695626

# Find important words

In [8]:
li_word_count = []
for elt in res.values:
    for item in elt:
        if type(item) == str:
            li_word_count.extend([unidecode.unidecode(x.lower()) for x in re.findall(r'\w+', item)])

In [9]:
dic_word_count = {}
for i in li_word_count:
    if len(i) > 0:
        if i in dic_word_count.keys():
            dic_word_count[i] +=1
        else:
            dic_word_count[i] = 1

In [10]:
df_word_count = pd.DataFrame.from_dict(dic_word_count, orient="index")
df_word_count.sort_values(0, ascending=False)
df_word_count.columns = ["freq_in_debate"]

In [11]:
del li_word_count
del dic_word_count

gc.collect()

202

In [12]:
df_word_count = df_word_count[df_word_count["freq_in_debate"] > 100]

In [13]:
li_stopwords = [unidecode.unidecode(x.strip().lower()) for x in open("french_stopwords.txt", 'r').readlines()]
li_stopwords = set(li_stopwords)
li_stopwords.remove("aucun")
li_stopwords.remove("tout")
li_stopwords.remove("tous")
li_stopwords.remove("non")

In [14]:
new_data = []
for i, elt in enumerate(res.index):
    temp_res = []
    for col in res.columns:
        item = res.loc[elt, col]
        if type(item) == str:
            li_item = [unidecode.unidecode(x.lower()) for x in re.findall(r'\w+', item) if unidecode.unidecode(x.lower()) not in li_stopwords]
            temp_res.append(li_item)
        else:
            temp_res.append(None)
    new_data.append(temp_res)

In [15]:
new_res = pd.DataFrame(new_data, columns=res.columns).to_sparse()
del new_data
gc.collect()

0

# Question 1 : Quelles sont toutes les choses qui pourraient être faites pour améliorer l'information des citoyens sur l'utilisation des impôts ?

In [16]:
question_1 = "Quelles sont toutes les choses qui pourraient être faites pour améliorer l\'information des citoyens sur l\'utilisation des impôts ?"
qu_1,df_qu_1_one_hot,df_cov_1,df_confidence_1,df_lift_1 = compute_similarities(question_1,new_res)

In [17]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_1[i].sort_values(ascending=False).head(10))

impots


impots         12019
utilisation     3299
depenses        2161
tous            1738
impot           1587
citoyens        1511
faire           1393
taxes           1344
site            1250
information     1248
Name: impots, dtype: uint16

depenses


depenses        10173
impots           2161
recettes         1990
publiques        1927
tous             1441
transparence     1349
site             1153
citoyens         1107
impot            1086
faire            1060
Name: depenses, dtype: uint16

impot


...

comptes         2797
cour            1360
depenses         687
tous             496
rapport          459
impots           410
citoyens         369
transparence     348
utilisation      346
faire            339
Name: comptes, dtype: uint16

repartition


repartition    2700
impots         1169
depenses        939
impot           607
utilisation     423
annee           324
tous            304
budget          303
publiques       271
information     269
Name: repartition, dtype: uint16

recettes


recettes    2476
depenses    1990
impots       552
budget       394
tous         391
annee        343
impot        333
bilan        297
site         293
citoyens     272
Name: recettes, dtype: uint16

In [18]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_1[i].sort_values(ascending=False).head(10))

impots


impots         1.000000
utilises       0.769723
servent        0.757611
collectes      0.752727
payes          0.747454
taxes          0.666006
gouv           0.648616
locaux         0.624288
grace          0.598746
utilisation    0.585240
Name: impots, dtype: float32

depenses


depenses         1.000000
recettes         0.803716
publiques        0.680198
inutiles         0.629344
postes           0.543260
tableau          0.513263
evolution        0.507389
secteur          0.482587
territoriales    0.475336
ministeres       0.475285
Name: depenses, dtype: float32

impot


...

comptes        1.000000
cour           0.962491
rapports       0.627957
cours          0.351039
rapport        0.275841
controle       0.264220
pouvoir        0.210879
resultats      0.194853
publication    0.161169
publics        0.160325
Name: comptes, dtype: float32

repartition


repartition    1.000000
1000           0.243101
camembert      0.242857
declaration    0.144074
annuelle       0.143523
tableau        0.143236
imposition     0.142364
indiquer       0.140518
avis           0.128486
postes         0.123742
Name: repartition, dtype: float32

recettes


recettes       1.000000
fiscales       0.255276
depenses       0.195616
tableau        0.184350
affectation    0.172911
bilan          0.156398
dette          0.155477
postes         0.135815
tva            0.132444
publier        0.120000
Name: recettes, dtype: float32

In [19]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_1[i].sort_values(ascending=False).head(10))

impots


impots         0.000083
utilises       0.000064
servent        0.000063
collectes      0.000063
payes          0.000062
taxes          0.000055
gouv           0.000054
locaux         0.000052
grace          0.000050
utilisation    0.000049
Name: impots, dtype: float32

depenses


depenses         0.000098
recettes         0.000079
publiques        0.000067
inutiles         0.000062
postes           0.000053
tableau          0.000050
evolution        0.000050
secteur          0.000047
territoriales    0.000047
ministeres       0.000047
Name: depenses, dtype: float32

impot


...

comptes        0.000358
cour           0.000344
rapports       0.000225
cours          0.000126
rapport        0.000099
controle       0.000094
pouvoir        0.000075
resultats      0.000070
publication    0.000058
publics        0.000057
Name: comptes, dtype: float32

repartition


repartition    0.000370
1000           0.000090
camembert      0.000090
declaration    0.000053
annuelle       0.000053
tableau        0.000053
imposition     0.000053
indiquer       0.000052
avis           0.000048
postes         0.000046
Name: repartition, dtype: float32

recettes


recettes       0.000404
fiscales       0.000103
depenses       0.000079
tableau        0.000074
affectation    0.000070
bilan          0.000063
dette          0.000063
postes         0.000055
tva            0.000053
publier        0.000048
Name: recettes, dtype: float32

In [20]:
gc.collect()

147

# Question 2 : Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?

In [21]:
question_2 = "Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?"
qu_2,df_qu_2_one_hot,df_cov_2,df_confidence_2,df_lift_2 = compute_similarities(question_2,new_res)

In [22]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_2[i].sort_values(ascending=False).head(10))

impot


impot       21249
revenu       9823
tous         7227
revenus      7049
tout         6790
payer        5561
impots       5101
monde        4762
fiscales     4287
faire        4148
Name: impot, dtype: uint16

impots


impots     13706
impot       5101
tout        4355
payer       4314
tous        4300
revenus     4254
revenu      3938
monde       3055
faire       2966
france      2740
Name: impots, dtype: uint16

revenus


...

france         6073
impot          3036
impots         2740
payer          2078
tous           2054
entreprises    2038
tout           1961
revenus        1907
fiscale        1894
francais       1840
Name: france, dtype: uint16

taxer


taxer          5751
impot          1931
revenus        1853
entreprises    1693
fiscale        1435
impots         1405
tous           1193
france         1160
revenu         1158
tout           1138
Name: taxer, dtype: uint16

francais


francais    5667
impot       3330
tous        2555
impots      2248
revenu      2002
revenus     1858
tout        1854
payer       1854
france      1840
faire       1556
Name: francais, dtype: uint16

In [23]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_2[i].sort_values(ascending=False).head(10))

impot


impot            1.000000
progressif       0.831569
direct           0.825613
echappent        0.821818
echapper         0.811069
fortune          0.805589
proportionnel    0.798883
43               0.791667
revenu           0.783458
minime           0.777580
Name: impot, dtype: float32

impots


impots         1.000000
indirects      0.947692
directs        0.899721
progressifs    0.763514
credits        0.752315
locaux         0.718901
fonciers       0.661184
payes          0.660465
injustes       0.586592
reductions     0.573850
Name: impots, dtype: float32

revenus


...

france        1.000000
realise       0.647239
etranger      0.632046
sportifs      0.621262
etrangeres    0.592593
realises      0.580769
usa           0.570149
chiffre       0.556818
residents     0.541667
obliger       0.541126
Name: france, dtype: float32

taxer


taxer           1.000000
transactions    0.520112
speculation     0.481967
gafa            0.470555
lourdement      0.446991
financieres     0.438169
profits         0.433628
fortement       0.397192
davantage       0.357091
dividendes      0.350497
Name: taxer, dtype: float32

francais


francais       1.000000
etranger       0.528816
residents      0.513889
etrangers      0.482890
usa            0.480597
territoire     0.440816
nationalite    0.430020
vivant         0.422287
43             0.416667
moitie         0.377386
Name: francais, dtype: float32

In [24]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_2[i].sort_values(ascending=False).head(10))

impot


impot            0.000047
progressif       0.000039
direct           0.000039
echappent        0.000039
echapper         0.000038
fortune          0.000038
proportionnel    0.000038
43               0.000037
revenu           0.000037
minime           0.000037
Name: impot, dtype: float32

impots


impots         0.000073
indirects      0.000069
directs        0.000066
progressifs    0.000056
credits        0.000055
locaux         0.000052
fonciers       0.000048
payes          0.000048
injustes       0.000043
reductions     0.000042
Name: impots, dtype: float32

revenus


...

france        0.000165
realise       0.000107
etranger      0.000104
sportifs      0.000102
etrangeres    0.000098
realises      0.000096
usa           0.000094
chiffre       0.000092
residents     0.000089
obliger       0.000089
Name: france, dtype: float32

taxer


taxer           0.000174
transactions    0.000090
speculation     0.000084
gafa            0.000082
lourdement      0.000078
financieres     0.000076
profits         0.000075
fortement       0.000069
davantage       0.000062
dividendes      0.000061
Name: taxer, dtype: float32

francais


francais       0.000176
etranger       0.000093
residents      0.000091
etrangers      0.000085
usa            0.000085
territoire     0.000078
nationalite    0.000076
vivant         0.000075
43             0.000074
moitie         0.000067
Name: francais, dtype: float32

In [25]:
gc.collect()

105

# Question 3 : Quels sont selon vous les impôts qu'il faut baisser en priorité ?

In [26]:
question_3 = "Quels sont selon vous les impôts qu\'il faut baisser en priorité ?"
qu_3,df_qu_3_one_hot,df_cov_3,df_confidence_3,df_lift_3 = compute_similarities(question_3, new_res)

In [27]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_3[i].sort_values(ascending=False).head(10))

tva


tva          17529
produits      8657
necessite     6110
impot         2606
impots        2190
taxe          2130
csg           2108
taxes         1847
revenu        1796
baisser       1684
Name: tva, dtype: uint16

impots


impots     12060
revenu      3166
baisser     2305
tva         2190
impot       1830
faut        1805
revenus     1796
tous        1692
taxe        1605
taxes       1572
Name: impots, dtype: uint16

impot


...

fonciere      2479
taxe          2349
habitation    1200
impot          583
tva            554
revenu         524
impots         501
taxes          376
supprimer      308
tous           279
Name: fonciere, dtype: uint16

augmenter


augmenter    2247
tva          1073
produits      853
baisser       786
impots        732
impot         601
necessite     563
faut          554
luxe          487
revenu        394
Name: augmenter, dtype: uint16

baisse


baisse       2209
tva           932
impots        801
impot         694
produits      626
necessite     450
taxe          401
faut          375
tous          348
revenu        343
Name: baisse, dtype: uint16

In [28]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_3[i].sort_values(ascending=False).head(10))

tva


tva             1.000000
luxe            0.918097
legumes         0.896552
1ere            0.888660
alimentaires    0.887365
necessite       0.886406
produits        0.879240
bio             0.875776
courante        0.832192
necessites      0.818620
Name: tva, dtype: float32

impots


impots           1.000000
indirects        0.953157
fonciers         0.807099
locaux           0.807081
injustes         0.642076
production       0.598958
publics          0.559896
ensuite          0.519084
forcement        0.515464
collectivites    0.514368
Name: impots, dtype: float32

impot


...

fonciere         1.000000
proprietaires    0.432177
principale       0.287190
taxe             0.282128
residence        0.263529
habitation       0.205903
immobilier       0.192171
logement         0.177966
succession       0.119134
droits           0.116301
Name: fonciere, dtype: float32

augmenter


augmenter    1.000000
luxe         0.379875
nombre       0.192568
diminuer     0.188295
riches       0.175599
achat        0.164557
pouvoir      0.161583
faudrait     0.157750
baisser      0.151416
prix         0.148098
Name: augmenter, dtype: float32

baisse


baisse          1.000000
hausse          0.227656
augmentation    0.211538
prix            0.202446
depenses        0.173845
achat           0.166365
pouvoir         0.165705
question        0.144754
suppression     0.134153
fiscale         0.133829
Name: baisse, dtype: float32

In [29]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_3[i].sort_values(ascending=False).head(10))

tva


tva             0.000057
luxe            0.000052
legumes         0.000051
1ere            0.000051
alimentaires    0.000051
necessite       0.000051
produits        0.000050
bio             0.000050
courante        0.000047
necessites      0.000047
Name: tva, dtype: float32

impots


impots           0.000083
indirects        0.000079
fonciers         0.000067
locaux           0.000067
injustes         0.000053
production       0.000050
publics          0.000046
ensuite          0.000043
forcement        0.000043
collectivites    0.000043
Name: impots, dtype: float32

impot


...

fonciere         0.000403
proprietaires    0.000174
principale       0.000116
taxe             0.000114
residence        0.000106
habitation       0.000083
immobilier       0.000078
logement         0.000072
succession       0.000048
droits           0.000047
Name: fonciere, dtype: float32

augmenter


augmenter    0.000445
luxe         0.000169
nombre       0.000086
diminuer     0.000084
riches       0.000078
achat        0.000073
pouvoir      0.000072
faudrait     0.000070
baisser      0.000067
prix         0.000066
Name: augmenter, dtype: float32

baisse


baisse          0.000453
hausse          0.000103
augmentation    0.000096
prix            0.000092
depenses        0.000079
achat           0.000075
pouvoir         0.000075
question        0.000066
suppression     0.000061
fiscale         0.000061
Name: baisse, dtype: float32

In [30]:
gc.collect()

105

# Question 4 : Afin de financer les dépenses sociales, faut-il selon vous... 

In [31]:
question_4 = "Afin de financer les dépenses sociales, faut-il selon vous..."
qu_4,df_qu_4_one_hot,df_cov_4,df_confidence_4,df_lift_4 = compute_similarities(question_4, new_res)

In [32]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_4[i].sort_values(ascending=False).head(10))

sociales


sociales       38105
aides          36861
revoir         36303
attribution    36254
conditions     36250
augmenter      14958
travail        14630
temps          13873
retraite       11689
age            11096
Name: sociales, dtype: uint16

aides


aides          37482
sociales       36861
revoir         36301
attribution    36278
conditions     36254
augmenter      14826
travail        14537
temps          13850
retraite       11620
age            11063
Name: aides, dtype: uint16

revoir


...

supprimer      2521
aides          1445
sociales       1440
revoir         1334
conditions     1302
attribution    1293
augmenter       764
travail         702
retraite        650
temps           626
Name: supprimer, dtype: uint16

reduire


reduire        2508
sociales       1356
aides          1298
revoir         1245
attribution    1224
conditions     1223
depenses        708
travail         700
augmenter       695
temps           656
Name: reduire, dtype: uint16

tous


tous           2282
sociales       1314
aides          1268
revoir         1233
conditions     1200
attribution    1184
travail         771
retraite        738
augmenter       690
temps           623
Name: tous, dtype: uint16

In [33]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_4[i].sort_values(ascending=False).head(10))

sociales


sociales         1.000000
attribution      0.997194
conditions       0.995414
aides            0.983432
revoir           0.983102
prestations      0.860911
rsa              0.854271
etrangers        0.813758
beneficiaires    0.811209
allocation       0.806780
Name: sociales, dtype: float32

aides


aides            1.000000
attribution      0.997855
conditions       0.995524
revoir           0.983048
sociales         0.967353
rsa              0.864322
etrangers        0.842282
beneficiaires    0.837758
controles        0.817857
enfants          0.816764
Name: aides, dtype: float32

revoir


...

supprimer     1.000000
niches        0.454759
presidents    0.397163
speciaux      0.382456
anciens       0.362256
fiscales      0.313804
regimes       0.291755
avantages     0.288740
privileges    0.287263
deputes       0.268293
Name: supprimer, dtype: float32

reduire


reduire            1.000000
reserver           0.596154
80                 0.481481
immigration        0.474934
train              0.473815
an                 0.434579
milliards          0.300987
multinationales    0.297994
fonctionnement     0.295276
cice               0.294212
Name: reduire, dtype: float32

tous


tous         1.000000
regimes      0.310782
citoyens     0.267631
avantages    0.199554
deputes      0.193380
francais     0.177227
revenu       0.176938
elus         0.173262
ans          0.171014
retraites    0.155267
Name: tous, dtype: float32

In [34]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_4[i].sort_values(ascending=False).head(10))

sociales


sociales         0.000026
attribution      0.000026
conditions       0.000026
aides            0.000026
revoir           0.000026
prestations      0.000023
rsa              0.000022
etrangers        0.000021
beneficiaires    0.000021
allocation       0.000021
Name: sociales, dtype: float32

aides


aides            0.000027
attribution      0.000027
conditions       0.000027
revoir           0.000026
sociales         0.000026
rsa              0.000023
etrangers        0.000022
beneficiaires    0.000022
controles        0.000022
enfants          0.000022
Name: aides, dtype: float32

revoir


...

supprimer     0.000397
niches        0.000180
presidents    0.000158
speciaux      0.000152
anciens       0.000144
fiscales      0.000124
regimes       0.000116
avantages     0.000115
privileges    0.000114
deputes       0.000106
Name: supprimer, dtype: float32

reduire


reduire            0.000399
reserver           0.000238
80                 0.000192
immigration        0.000189
train              0.000189
an                 0.000173
milliards          0.000120
multinationales    0.000119
fonctionnement     0.000118
cice               0.000117
Name: reduire, dtype: float32

tous


tous         0.000438
regimes      0.000136
citoyens     0.000117
avantages    0.000087
deputes      0.000085
francais     0.000078
revenu       0.000078
elus         0.000076
ans          0.000075
retraites    0.000068
Name: tous, dtype: float32

In [35]:
gc.collect()

105

# Question 5 : S'il faut selon vous revoir les conditions d'attribution de certaines aides sociales, lesquelles doivent être concernées ?

In [36]:
question_5 = "S'il faut selon vous revoir les conditions d\'attribution de certaines aides sociales, lesquelles doivent être concernées ?"
qu_5,df_qu_5_one_hot,df_cov_5,df_confidence_5,df_lift_5 = compute_similarities(question_5, new_res)

In [37]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_5[i].sort_values(ascending=False).head(10))

aides


aides          15338
sociales        6053
allocations     2561
faut            2382
familiales      2164
aide            2130
enfants         1918
chomage         1902
travail         1812
revenus         1521
Name: aides, dtype: uint16

allocations


allocations    11198
familiales      8127
chomage         2798
enfants         2768
aides           2561
revenus         1748
rsa             1579
aide            1223
enfant          1205
familles        1137
Name: allocations, dtype: uint16

familiales


...

revenu         2717
aides          1314
allocations     819
familiales      690
sociales        659
universel       616
aide            508
enfants         410
revenus         397
faut            386
Name: revenu, dtype: uint16

revoir


revoir         2692
aides          1336
attribution     790
faut            729
conditions      710
sociales        651
allocations     644
chomage         551
familiales      484
aide            431
Name: revoir, dtype: uint16

conditions


conditions     2567
aides          1211
attribution    1082
revoir          710
sociales        680
allocations     674
familiales      563
faut            505
chomage         446
ressources      443
Name: conditions, dtype: uint16

In [38]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_5[i].sort_values(ascending=False).head(10))

aides


aides         1.000000
sociales      0.908041
regrouper     0.782759
cumul         0.758621
accordees     0.725166
ensemble      0.677994
gestion       0.657480
attribuees    0.649526
donnees       0.644706
vivre         0.629596
Name: aides, dtype: float32

allocations


allocations     1.000000
familiales      0.916234
plafonnees      0.742947
chomages        0.731111
aisees          0.659574
enfant          0.607053
plafonnement    0.593525
4               0.588235
aises           0.574586
3               0.569514
Name: allocations, dtype: float32

familiales


...

revenu        1.000000
universel     0.908555
fiscal        0.395018
base          0.380165
impot         0.369760
minimum       0.259762
seuil         0.247984
foyer         0.231377
solidarite    0.222958
niveau        0.207675
Name: revenu, dtype: float32

revoir


revoir          1.000000
attribution     0.279745
attributions    0.279026
conditions      0.276587
baisse          0.238411
politique       0.195122
systeme         0.167564
faut            0.157044
faudrait        0.150888
sociaux         0.118291
Name: revoir, dtype: float32

conditions


conditions      1.000000
attribution     0.383144
ressources      0.381239
attributions    0.368914
revoir          0.263744
acces           0.184725
faudrait        0.132150
mettre          0.124736
minimum         0.112903
faut            0.108789
Name: conditions, dtype: float32

In [39]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_5[i].sort_values(ascending=False).head(10))

aides


aides         0.000065
sociales      0.000059
regrouper     0.000051
cumul         0.000049
accordees     0.000047
ensemble      0.000044
gestion       0.000043
attribuees    0.000042
donnees       0.000042
vivre         0.000041
Name: aides, dtype: float32

allocations


allocations     0.000089
familiales      0.000082
plafonnees      0.000066
chomages        0.000065
aisees          0.000059
enfant          0.000054
plafonnement    0.000053
4               0.000053
aises           0.000051
3               0.000051
Name: allocations, dtype: float32

familiales


...

revenu        0.000368
universel     0.000334
fiscal        0.000145
base          0.000140
impot         0.000136
minimum       0.000096
seuil         0.000091
foyer         0.000085
solidarite    0.000082
niveau        0.000076
Name: revenu, dtype: float32

revoir


revoir          0.000371
attribution     0.000104
attributions    0.000104
conditions      0.000103
baisse          0.000089
politique       0.000072
systeme         0.000062
faut            0.000058
faudrait        0.000056
sociaux         0.000044
Name: revoir, dtype: float32

conditions


conditions      0.000390
attribution     0.000149
ressources      0.000149
attributions    0.000144
revoir          0.000103
acces           0.000072
faudrait        0.000051
mettre          0.000049
minimum         0.000044
faut            0.000042
Name: conditions, dtype: float32

In [40]:
gc.collect()

105

# Question 6 : Quels sont les domaines prioritaires où notre protection sociale doit être renforcée ?

In [41]:
question_6 = "Quels sont les domaines prioritaires où notre protection sociale doit être renforcée ?"
qu_6,df_qu_6_one_hot,df_cov_6,df_confidence_6,df_lift_6 = compute_similarities(question_6, new_res)

In [42]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_6[i].sort_values(ascending=False).head(10))

sante


sante         14508
education      2012
logement       1115
retraite        986
agees           647
tous            645
retraites       644
soins           627
dependance      617
hopitaux        588
Name: sante, dtype: uint16

agees


agees          4422
aide            876
sante           647
dependantes     537
dependance      523
handicapes      502
enfants         471
charge          396
aides           376
handicapees     374
Name: agees, dtype: uint16

aide


...

charge        1926
prise         1264
sante          518
agees          396
soins          278
enfants        238
sociale        236
dependance     231
aide           187
retraite       181
Name: charge, dtype: uint16

faut


faut          1925
sociale        429
sante          402
protection     364
faire          270
tous           246
tout           243
aides          222
aide           213
systeme        197
Name: faut, dtype: uint16

vie


vie          1856
fin           382
retraites     379
sante         366
retraite      353
aide          257
agees         252
tous          224
tout          212
sociale       211
Name: vie, dtype: uint16

In [43]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_6[i].sort_values(ascending=False).head(10))

sante


sante         1.000000
education     0.666667
justice       0.584577
publique      0.556291
prevention    0.494845
medecins      0.484502
soigner       0.484472
ecole         0.468421
domaine       0.456491
nationale     0.444015
Name: sante, dtype: float32

agees


agees          1.000000
dependantes    0.710317
isolees        0.470588
handicapees    0.440000
domicile       0.427885
maintien       0.425170
ephad          0.316770
ehpad          0.296343
accueil        0.294798
maison         0.263158
Name: agees, dtype: float32

aide


...

charge       1.000000
prise        0.898365
prendre      0.373178
meilleure    0.298765
frais        0.210037
maladies     0.168558
dentaires    0.149425
ehpad        0.145019
soins        0.122305
age          0.110270
Name: charge, dtype: float32

faut


faut         1.000000
renforcer    0.295008
mettre       0.265700
systeme      0.199191
faire        0.198238
aider        0.196406
mieux        0.192708
depenses     0.187611
pouvoir      0.177444
moyens       0.173554
Name: faut, dtype: float32

vie


vie          1.000000
fin          0.697080
travaille    0.452261
cout         0.349914
pouvoir      0.201504
vivre        0.186978
tout         0.139199
aider        0.133504
ans          0.127396
retraites    0.123533
Name: vie, dtype: float32

In [44]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_6[i].sort_values(ascending=False).head(10))

sante


sante         0.000069
education     0.000046
justice       0.000040
publique      0.000038
prevention    0.000034
medecins      0.000033
soigner       0.000033
ecole         0.000032
domaine       0.000031
nationale     0.000031
Name: sante, dtype: float32

agees


agees          0.000226
dependantes    0.000161
isolees        0.000106
handicapees    0.000100
domicile       0.000097
maintien       0.000096
ephad          0.000072
ehpad          0.000067
accueil        0.000067
maison         0.000060
Name: agees, dtype: float32

aide


...

charge       0.000519
prise        0.000466
prendre      0.000194
meilleure    0.000155
frais        0.000109
maladies     0.000088
dentaires    0.000078
ehpad        0.000075
soins        0.000064
age          0.000057
Name: charge, dtype: float32

faut


faut         0.000519
renforcer    0.000153
mettre       0.000138
systeme      0.000103
faire        0.000103
aider        0.000102
mieux        0.000100
depenses     0.000097
pouvoir      0.000092
moyens       0.000090
Name: faut, dtype: float32

vie


vie          0.000539
fin          0.000376
travaille    0.000244
cout         0.000189
pouvoir      0.000109
vivre        0.000101
tout         0.000075
aider        0.000072
ans          0.000069
retraites    0.000067
Name: vie, dtype: float32

In [45]:
gc.collect()

105

# Question 7 : Pour quelle(s) politique(s) publique(s) ou pour quels domaines d'action publique, seriez-vous prêts à payer plus d'impôts ?

In [46]:
question_7 = "Pour quelle(s) politique(s) publique(s) ou pour quels domaines d\'action publique, seriez-vous prêts à payer plus d\'impôts ?"
qu_7,df_qu_7_one_hot,df_cov_7,df_confidence_7,df_lift_7 = compute_similarities(question_7, new_res)

In [47]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_7[i].sort_values(ascending=False).head(10))

sante


sante            8094
education        3389
securite          934
justice           651
recherche         470
environnement     370
ecologie          306
hopitaux          297
impots            295
publique          291
Name: sante, dtype: uint16

education


education        7164
sante            3389
justice           691
securite          682
recherche         637
nationale         637
formation         385
environnement     381
transition        315
ecologie          313
Name: education, dtype: uint16

aucun


...

recherche        1731
education         637
sante             470
enseignement      144
transition        140
justice           129
securite          123
ecologique        120
developpement     118
defense           105
Name: recherche, dtype: uint16

france


france    1600
impots     558
pays       344
aucun      337
faut       293
payer      279
impot      232
faire      214
monde      210
tout       157
Name: france, dtype: uint16

argent


argent    1568
impots     521
payer      382
aucun      372
faut       337
mieux      299
public     250
faire      217
impot      198
tout       169
Name: argent, dtype: uint16

In [48]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_7[i].sort_values(ascending=False).head(10))

sante


sante        1.000000
education    0.473060
securite     0.377832
culture      0.330724
justice      0.325012
personnel    0.324324
ecole        0.320225
armee        0.317259
police       0.316411
transport    0.298050
Name: sante, dtype: float32

education


education          1.000000
nationale          0.697700
culture            0.569472
sante              0.418705
formation          0.402720
professionnelle    0.376384
recherche          0.367995
justice            0.344983
jeunes             0.320000
enfants            0.317212
Name: education, dtype: float32

aucun


...

recherche        1.000000
enseignement     0.222222
developpement    0.187302
defense          0.106168
education        0.088917
transition       0.069825
ecologique       0.066225
justice          0.064403
sante            0.058068
securite         0.049757
Name: recherche, dtype: float32

france


france          1.000000
prelevements    0.283721
pays            0.260606
monde           0.201536
francais        0.155637
niveau          0.139896
augmenter       0.112830
fiscale         0.110874
faire           0.105108
faut            0.101209
Name: france, dtype: float32

argent


argent      1.000000
public      0.323415
mal         0.242798
depense     0.194495
mieux       0.165468
francais    0.123775
faut        0.116408
faire       0.106582
tout        0.094203
payer       0.089419
Name: argent, dtype: float32

In [49]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_7[i].sort_values(ascending=False).head(10))

sante


sante        0.000124
education    0.000058
securite     0.000047
culture      0.000041
justice      0.000040
personnel    0.000040
ecole        0.000040
armee        0.000039
police       0.000039
transport    0.000037
Name: sante, dtype: float32

education


education          0.000140
nationale          0.000097
culture            0.000079
sante              0.000058
formation          0.000056
professionnelle    0.000053
recherche          0.000051
justice            0.000048
jeunes             0.000045
enfants            0.000044
Name: education, dtype: float32

aucun


...

recherche        0.000578
enseignement     0.000128
developpement    0.000108
defense          0.000061
education        0.000051
transition       0.000040
ecologique       0.000038
justice          0.000037
sante            0.000034
securite         0.000029
Name: recherche, dtype: float32

france


france          0.000625
prelevements    0.000177
pays            0.000163
monde           0.000126
francais        0.000097
niveau          0.000087
augmenter       0.000071
fiscale         0.000069
faire           0.000066
faut            0.000063
Name: france, dtype: float32

argent


argent      0.000638
public      0.000206
mal         0.000155
depense     0.000124
mieux       0.000106
francais    0.000079
faut        0.000074
faire       0.000068
tout        0.000060
payer       0.000057
Name: argent, dtype: float32

In [50]:
gc.collect()

126

# Question 8 : Y a-t-il d'autres points sur les impôts et les dépenses sur lesquels vous souhaiteriez vous exprimer ?

In [51]:
question_8 = "Y a-t-il d\'autres points sur les impôts et les dépenses sur lesquels vous souhaiteriez vous exprimer ?"
qu_8,df_qu_8_one_hot,df_cov_8,df_confidence_8,df_lift_8 = compute_similarities(question_8, new_res)

In [52]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_8[i].sort_values(ascending=False).head(10))

impots


impots         8014
payer          2086
faut           1937
faire          1910
tous           1902
tout           1856
impot          1843
france         1744
depenses       1587
entreprises    1304
Name: impots, dtype: uint16

faut


faut         7863
faire        2215
impots       1937
tous         1869
tout         1817
depenses     1811
impot        1587
france       1523
supprimer    1351
vie          1287
Name: faut, dtype: uint16

tous


...

francais    4110
france      1493
tous        1382
impots      1286
faire       1281
tout        1185
impot       1169
faut        1161
pays         947
payer        902
Name: francais, dtype: uint16

revenus


revenus    4074
impot      1495
tous       1307
impots     1288
tout       1028
faire       980
revenu      939
faut        933
france      828
payer       784
Name: revenus, dtype: uint16

pays


pays        4029
france      1582
faire       1274
tous        1224
faut        1174
impots      1128
tout        1113
depenses    1001
francais     947
impot        922
Name: pays, dtype: uint16

In [53]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_8[i].sort_values(ascending=False).head(10))

impots


impots     1.000000
credits    0.562264
payent     0.539519
payons     0.528736
paient     0.514620
locaux     0.492754
paie       0.485313
payant     0.454902
taxes      0.445946
payer      0.434131
Name: impots, dtype: float32

faut


faut               1.000000
indecents          0.573529
longue             0.560907
liste              0.511002
theodule           0.501577
bareme             0.495356
assemblees         0.489362
courage            0.467577
recommandations    0.467391
progressif         0.464503
Name: faut, dtype: float32

tous


...

francais      1.000000
etranger      0.443756
etrangers     0.426376
francaises    0.371681
territoire    0.354839
francaise     0.354565
majorite      0.342857
jaunes        0.321337
9             0.321212
80            0.317891
Name: francais, dtype: float32

revenus


revenus          1.000000
faibles          0.526163
imposables       0.459732
modestes         0.448387
tranche          0.435772
progressivite    0.433460
immobiliers      0.431298
capital          0.428803
declaration      0.413174
tranches         0.403321
Name: revenus, dtype: float32

pays


pays             1.000000
europeens        0.583882
origine          0.513619
europe           0.435967
allemagne        0.403189
etrangers        0.391805
europeen         0.367491
pib              0.363636
developpement    0.362543
paradis          0.354128
Name: pays, dtype: float32

In [54]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_8[i].sort_values(ascending=False).head(10))

impots


impots     0.000125
credits    0.000070
payent     0.000067
payons     0.000066
paient     0.000064
locaux     0.000061
paie       0.000061
payant     0.000057
taxes      0.000056
payer      0.000054
Name: impots, dtype: float32

faut


faut               0.000127
indecents          0.000073
longue             0.000071
liste              0.000065
theodule           0.000064
bareme             0.000063
assemblees         0.000062
courage            0.000059
recommandations    0.000059
progressif         0.000059
Name: faut, dtype: float32

tous


...

francais      0.000243
etranger      0.000108
etrangers     0.000104
francaises    0.000090
territoire    0.000086
francaise     0.000086
majorite      0.000083
jaunes        0.000078
9             0.000078
80            0.000077
Name: francais, dtype: float32

revenus


revenus          0.000245
faibles          0.000129
imposables       0.000113
modestes         0.000110
tranche          0.000107
progressivite    0.000106
immobiliers      0.000106
capital          0.000105
declaration      0.000101
tranches         0.000099
Name: revenus, dtype: float32

pays


pays             0.000248
europeens        0.000145
origine          0.000127
europe           0.000108
allemagne        0.000100
etrangers        0.000097
europeen         0.000091
pib              0.000090
developpement    0.000090
paradis          0.000088
Name: pays, dtype: float32

In [55]:
gc.collect()

98